### predictions

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

from functions.model_data import bestHyperparamaters 
from functions.postproccessing import process_results

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats
%store -r X_test_labeled_df



In [3]:
train_data_formodel.head()

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.630000,943.71,464944.18,50.280000,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05,0
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,43.735431,1077.11,465010.94,70.910000,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04,0
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.700000,620.21,465295.41,104.470000,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05,0
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,0.73,47.06,162.420000,1143.19,402596.61,3735.154885,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03,0
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.520000,547.22,465027.11,44.440000,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,0


### configurations

* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [4]:
save_predictions = True

In [5]:
from config_and_dependencies.config import *

### counts of instances in all classes before oversampling

In [6]:
train_data_formodel['class'].value_counts()

4     105
15    100
16     61
0      53
11     45
13     36
14     36
2      36
10     30
7      30
6      30
5      27
8      27
1      24
12     21
3      18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y and the variables identified as best by the 2 feature_selection notebook are used as features

In [7]:
y = np.array(train_data_formodel[target])
X = np.array(np.array(train_data_formodel[best_feats]))

### the dimensions of the class and features are checked

In [8]:
print(X.shape)
print(y.shape)

(696, 15)
(696,)


### grid search is utilised to identify optimum hyperparamaters

In [ ]:
my_optimiser = bestHyperparamaters(X, y)
my_optimiser.get_best_model()

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   54.9s


In [14]:
best_model = my_optimiser.best_model

### Model is built for predicting source of artefacts 

In [15]:
best_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2400, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

# sanity check to see model correctly predicts geological samples

In [18]:
train_check = best_model.predict(X)
sum(train_check == y)/len(y)

1.0

In [20]:
X.shape

(696, 15)

# When I check the encodings below with that in notebook 1, it is to be correct

In [21]:
train_check

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [22]:
print(best_model)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2400, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)


In [23]:
import pickle


In [24]:
#filename = 'output_datasets/models/rfc_model.sav'
#pickle.dump(best_model, open(filename, 'wb'))


In [25]:
my_dict = {}
numbers = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]
sites = ['FH', 'ER', 'WW', 'TC', 'CS', 'KQ', 'AR', 'SL', 'FG', 'WB', 'PF', 'WH',
       'SQ', 'WN', 'BH', 'PH', 'LB']

In [26]:
for i in numbers:
    my_dict[i] = sites[i]

In [27]:
my_dict

{0: 'FH',
 1: 'ER',
 2: 'WW',
 3: 'TC',
 4: 'CS',
 5: 'KQ',
 6: 'AR',
 7: 'SL',
 8: 'FG',
 9: 'WB',
 10: 'PF',
 11: 'WH',
 12: 'SQ',
 13: 'WN',
 14: 'BH',
 15: 'PH',
 16: 'LB'}

In [28]:
print(train_data_formodel[target].unique())
print(uniques)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Index(['FH', 'ER', 'WW', 'TC', 'CS', 'KQ', 'AR', 'SL', 'FG', 'WB', 'PF', 'WH',
       'SQ', 'WN', 'BH', 'PH', 'LB'],
      dtype='object')


In [29]:
identifiers =  X_test_labeled_df['Analysis']

In [30]:
results = process_results(best_model, X_test_labeled_df, best_feats, uniques, identifiers)

Pandas Apply: 100%|██████████| 363/363 [00:00<00:00, 29764.86it/s]


In [31]:
results.head()

,class_number,FH,ER,WW,TC,CS,KQ,AR,SL,FG,WB,PF,WH,SQ,WN,BH,PH,LB,Analysis,inlierLabel,class_predictions
0,9,0.041667,0.035000,0.177083,0.020000,0.003750,0.005833,0.000000,0.052083,0.019583,0.473333,0.0,0.016667,0.058750,0.011667,0.015833,0.066667,0.002083,06_DH1_1,1,WB
1,9,0.048750,0.034583,0.264583,0.009583,0.002500,0.064583,0.000833,0.024583,0.038333,0.317500,0.0,0.020000,0.071250,0.017917,0.027500,0.050000,0.007500,07_DH1_2,-1,other
2,9,0.048333,0.042917,0.253333,0.008750,0.002083,0.062917,0.001667,0.024167,0.039583,0.302500,0.0,0.020833,0.079167,0.020417,0.027500,0.058750,0.007083,08_DH1_3,-1,other
3,9,0.090833,0.033750,0.305833,0.010000,0.003333,0.013750,0.000000,0.036250,0.032083,0.341667,0.0,0.009167,0.037083,0.023333,0.009167,0.052917,0.000833,09_DH2_1,1,WB
4,2,0.104167,0.035417,0.442917,0.018750,0.014167,0.018333,0.000000,0.041250,0.037083,0.188333,0.0,0.008750,0.018333,0.015417,0.007083,0.049167,0.000833,10_DH2_2,1,WW


### I think if there is a problem here it's most likely to be in the encodings of the class, perhaps they are not correct?

In [32]:
results['class_number'].value_counts()

2    206
9    156
7      1
Name: class_number, dtype: int64

### predictions are stored as a variable into memory

In [33]:
%store results

Stored 'results' (DataFrame)


### predictions are outputted as csv file

In [34]:
if save_predictions:
    results.to_csv('output_datasets/predictions.csv')


In [35]:
results.head()

,class_number,FH,ER,WW,TC,CS,KQ,AR,SL,FG,WB,PF,WH,SQ,WN,BH,PH,LB,Analysis,inlierLabel,class_predictions
0,9,0.041667,0.035000,0.177083,0.020000,0.003750,0.005833,0.000000,0.052083,0.019583,0.473333,0.0,0.016667,0.058750,0.011667,0.015833,0.066667,0.002083,06_DH1_1,1,WB
1,9,0.048750,0.034583,0.264583,0.009583,0.002500,0.064583,0.000833,0.024583,0.038333,0.317500,0.0,0.020000,0.071250,0.017917,0.027500,0.050000,0.007500,07_DH1_2,-1,other
2,9,0.048333,0.042917,0.253333,0.008750,0.002083,0.062917,0.001667,0.024167,0.039583,0.302500,0.0,0.020833,0.079167,0.020417,0.027500,0.058750,0.007083,08_DH1_3,-1,other
3,9,0.090833,0.033750,0.305833,0.010000,0.003333,0.013750,0.000000,0.036250,0.032083,0.341667,0.0,0.009167,0.037083,0.023333,0.009167,0.052917,0.000833,09_DH2_1,1,WB
4,2,0.104167,0.035417,0.442917,0.018750,0.014167,0.018333,0.000000,0.041250,0.037083,0.188333,0.0,0.008750,0.018333,0.015417,0.007083,0.049167,0.000833,10_DH2_2,1,WW
